# Deriving cancer gene's from expression data

See if it's possible to train a deep neural network tumor/normal binary classifier using just the Toil TCGA, TARGET and GTEX expression datasets:

https://xenabrowser.net/datapages/?host=https://toil.xenahubs.net)

If we can, then see whether any of the early layers re-capitulates the COSMIC cancer gene census:

http://cancer.sanger.ac.uk/census/

In [1]:
import os
import numpy as np
import pandas as pd

# fix random seed for reproducibility
np.random.seed(42)

In [18]:
from keras.utils.io_utils import HDF5Matrix

input_file = "data/tumor_normal.h5"

# different size parameters if debugging or using full dataset
if os.getenv("DEBUG", "True") == "True":
    x_train = HDF5Matrix(input_file, "x_train", start=0, end=1000)
    x_test = HDF5Matrix(input_file, "x_test", start=0, end=200)
    y_train = HDF5Matrix(input_file, "y_train", start=0, end=1000)
    y_test = HDF5Matrix(input_file, "y_test", start=0, end=200)
    print("Training on partial dataset")
    epochs=1
    batch_size=256
else:
    x_train = HDF5Matrix(input_file, "x_train")
    x_test = HDF5Matrix(input_file, "x_test")
    y_train = HDF5Matrix(input_file, "y_train")
    y_test = HDF5Matrix(input_file, "y_test")
    print("Training on full dataset")
    epochs=32
    batch_size=128
    
print("x_train.shape:", x_train.shape, "epochs:", epochs, "batch_size:", batch_size)

Training on partial dataset
x_train.shape: (1000, 60498) epochs: 1 batch_size: 256


In [16]:
"""
Trivial network to just verify all the pieces are working
"""

from keras.layers import Dense
from keras.models import Model, Sequential

classify = [
    Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='sigmoid')
]

model = Sequential(classify)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, shuffle="batch")

Epoch 1/1
1000/1000 [==============================] - 3s - loss: 7.9554 - acc: 0.4630     
